# Ejemplos de Hands-On

Preparar un repositorio de funciones y ejemplos en CoppeliaSim que los equipos puedan usar como guía para crear sus programas. 

El repositorio tendrá un archivo con las funciones básicas para manejar un robot en CoppeliaSim y una carpeta con ejemplos que usan este archivo para implementar tareas básicas del robot. Asimismo, habrá una carpeta con Misiones que usan estas funciones para demostrar casos más complejos.

Al concluir, deberían existir:
1. Un (1) archivo llamado "HandsOn-sim.lua" con las Funciones definidas abajos
    Este archivo será importado con el comando `require` en el script principal del robot en LUA.
2. Una carpeta llamada "Ejemplos"
    En la carpeta estarán los documentos necesarios para demostrar los ejemplos definidos abajo.
    Los scripts de LUA en esta carpeta pudiera ser importados usando `require`, si es necesario.
3. Una carpeta llamada "Misiones"
    En la carpeta habrán documentos (scripts y/o escenas) necesarios para que alguien pueda preparar y correr las misiones definidas abajo.


## Funciones

Cada función hace una operación simple, ya sea respecto al robot o al hardware. 

El hardware es abstracto para el software. Es decir, las funciones para hacer algún movimiento (como Avanzar) no interactúan directamente con el hardware (las articulaciones, en el caso del simulador) sino a través de una función de bajo nivel que se puede modificar según las necesidades del robot. De esta manera Avanzar no tiene que saber, por ejemplo, cuántos motores usa el robot para hacerlo, ni cómo le llega el comando. 

Por ejemplo:

```Lua

-- Para que un vehículo avance con una velocidad vel_a:
function Avanzar(vel_a)
    Desplazar(vel_a, 0) -- solo avanza, no gira
end

-- Para girar a una velocidad fija:
function Girar(vel_g)
    Desplazar(0, vel_g)
end

-- Donde
-- Mueve el vehículo con las velocidades de avance (lineal) y de giro (angular) solicitadas 
function Desplazar(v,w)
    -- Calcula lo que necesita "pedírsele" a los motores para 
    -- avanzar con velocidad lineal (v) y girar con velocidad angular (w)

    -- Si el vehículo tiene configuracion diferencial:
    Velocidad_Diferencial(v,w)
    -- Si fuera un triciclo, se llamara a 
    -- Velocidad_Triciclo(v,w)
end

-- Entonces
function Velocidad_Diferencial(v, w):
    -- Calcula las velocidades de cada rueda acorde a dimensiones físicas
    vel_izq, vel_der = f(v,w, L_base, R_rueda)

    -- Envia las velocidades a cada motor
    MotorIzq(vel_izq)
    MotorDer(vel_der)
end

-- Pero segun el vehículo, pudiera tener que usarse otra como:
function Velocidad_Triciclo(v,w):
    -- Calcula las velocidades de manera acorde
    vel_ruedas, ang_timon = f(v,w, L_base, R_rueda, D_delantero)

    -- Envia las velocidades a cada motor
    Motor_Tracción(vel_ruedas)
    Posicion_Timon(ang_timon) -- Asumiendo Servo
end

-- Continuando la abstracción: el "motor" derecho 
-- puede que en realidad sea varias articulaciones funcionando juntas
function MotorDer(w):
    -- Asigna un tercio de la velocidad angular solicitada a las articulaciones.
    sim.setJointTargetVelocity(motor1a, w/3)
    sim.setJointTargetVelocity(motor1b, w/3)
    sim.setJointTargetVelocity(motor1c, w/3)

```



Siguiendo este ejemplo, las tareas que utiliza las funciones para desplazar el vehículo no se ven directamente afectadas por la implementación del hardware en el vehículo (configuración de motores, etc.)

En este caso, se asumirá que los vehículos no pueden medir la velocidad que llevan, por lo que en vez de usar términos de velocidad, esta se representará como un *porcentaje* de una velocidad máxima guardada como una CONSTANTE en el programa para su uso interno al calcular la velocidad que se le enviará a la articulación en el simulador. 

Esto quiere decir que en el código todos los valores referidos a velocidades se presentarán como un valor entre $[-1.0, 1.0]$ en representación de un porcentaje de la velocidad máxima que puede alcanzar el vehículo, los motores, etc. Las únicas funciones que pudieran usar velocidades directamente (e.g.: en m/s o rad/s) serían las que directamente envían el comando a la articulación (e.g.: `MotorDer()` arriba), y posiblemente las que calculan las velocidades particulares según la configuración del vehículo (como `Velocidad_Triciclo()` arriba), que pudieran tener que poner límites. 


Las funciones mínimas a implementar se muestran abajo. En estas, el término `v_pc` o `w_pc` representan un porcentaje de velocidad lineal ($v$) o velocidad angular ($\omega$), ya sea del robot o del eje de un rotor (motor/articulación). 
### Básicas

Las relacionadas a los motores, por ejemplo:
- `MotorDer(w_pc)`
- `MotorIzq(w_pc)`
- `VelocidadDiferencial(v_pc, w_pc)`
- `Desplazar(v_pc, w_pc)`
    - Ver comentario en ejemplo arriba
- `Avanzar(v_pc)`
- `Girar(w_pc)`
- `PosicionServo(ang)`
    - Usaría `sim.setJointTargetPosition()`
- `SubirBrazo()`
    - Sube (rota) el brazo del robot hasta su punto de Abierto.
- `BajarBrazo()`
    - Baja (rota) el brazo del robot hasta su punto de cerrado.

Las relacionadas a los sensores, por ejemplo:
- `LeerPresencia(sensor)`:
    - Produce un booleano si es detectado algo con un sensor de proximidad
- `LeerProximidad(sensor)`:
    - devuelve la lectura de proximidad
- `LeerLuz(sensor)`:
    - Devuelve la lectura de Intensidad promedio de un sensor de visión
- `LeerColor(sensor)`:
    - Devuelve el valor promedio de {R,G,B} de un sensor de visión
- `LeerImagen(sensor)`:
    - Devuelve la tabla con la imagen extraída de un sensor de visión

Las de utilidad para hacer conversiones, si son necesarias, como:
- `grados_a_radianes(grd)`

### Esenciales

Las relacionadas al seteo del script y preparar todo:
- `Inicializar()`
    - Se llamaría en `sysCall_init()` y declara todas las variables _globales_ que se usarían en las demás funciones (como dimensiones, máximos, ganancias, etc.).
- `Limpiar()`
    - Si es necesario.
    - Se llamaría en el `sysCall_cleanup()` al finalizar una simulación.

### Detalles

El documento con las funciones solo debería tener funciones básicas y variables que el usuario pueda llamar dentro de `sysCall_actuation()` o `sysCall_sensing()` desde su script en CoppeliaSim.

Todas las variables definidas en `Inicializar()` son _globales_ y estarán disponibles para el resto del script. Si el usuario necesita sobre escribirlas, pudiera hacerlo dentro de `sysCall_init()` después que se llame a `Inicializar()`. Todas las variables definidas dentro de las funciones estarán declaradas como _locales_  usando el indicador `local` al declararlas. Por ejemplo:

```Lua 
function Velocidad_Diferencial(v, w):
    -- Calcula las velocidades de cada rueda acorde a dimensiones físicas
    local vel_izq, vel_der = f(v,w, L_base, R_rueda)

    -- Envia las velocidades a cada motor
    MotorIzq(vel_izq)
    MotorDer(vel_der)
end
```

De forma similar, las funciones como `MotorXXX` y `Desplazar` serán preparadas en base al robot provisto y estarán definidas en este archivo, pero pudieran ser sobreescritas por el usuario después de importar el archivo.

Para poder tener acceso a los resultados de cada una de las funciones, ya que las variables internas serían locales, cada función debe devolver cualquier valor relevante calculado internamente.

## Ejemplos 

En una carpeta aparte, llamada _Ejemplos_, estarán los documentos (scripts, escenas, modelos) para los siguientes ejemplos donde el robot: 

1. Se mueve hasta detectar algo oscuro en el piso.
2. Se mueve hasta una línea en el piso. 
3. Se mueve hasta cruzar X cantidad de líneas.
4. Se mueve hasta alcanzar la línea de X color entre varias otras.
5. Avanza hasta encontrarse a una distancia X de un objeto al frente.
6. Gira hasta detectar un objeto (al frente).
7. Gira en arcos ($v>0, \omega>0$) hasta detectar un objeto.

Idealmente, se pudiera utilizar una misma escena para varios de los ejemplos (o todos), y se puede cambiar entre ellos llamando diferentes funciones, o importando diferentes archivos.

## Misiones


Se prepararán dos (2) ejemplos funcionales en una carpeta llamada _Misiones_ que presentarán un vehículo haciendo algún tipo de misión usando las funciones disponibles. 

Las misiones son:

1. Luego de esperar 5 segundos, el robot se mueve para encontrar un objeto en la pista y se "alinea" con un anillo "pintado" en el piso que rodea el objeto desde alguna distancia. Una vez esté alineado, el vehículo avanza hasta el objeto y lo sujeta.

2. Luego de esperar 5 segundos, el robot busca, entre 4 objetos que se les presentan, un objeto de color X y lo intenta sujetar. Los 4 objetos estarían distribuidos alrededor del vehículo a una distancia arbitraria.

## Recursos

Se usarán los recuros en el repositorio de [https://github.com/iejim/hands-on-sim] en Github. En este se encuentra: 

1. Un ROVER básico con sensores para las funciones propuestas (en la Escena)
2. Una Pista para ROVER 2021 que puede servir de base (el tamaño de las cajas es más pequeño que lo regulatorio)  (en la Escena).
3. Un script `Rover21Src.lua` con las funciones para controlar el ROVER.
4. Un script `cam_thread.lua` con un procesamiento básico de imágenes para identificar objetos de colores.
5. Este archivo como un Jupyter Notebook (legible con Visual Studio Code).


El Robot es un robot diferencial (dos ruedas independientes) con una rueda loca. Cuenta con:

- Un motor independiente para cada rueda.
- Una rueda loca para balancear el robot, sin fricción.
- Tres sensores de línea capaces de ver color.
- Un sensor de color para detectar el color de una caja en su control.
- Un sensor de proximidad para encontrar objetos frente al robot.
- Una cámara con su propio procesamiento de imágen.
- Un brazo controlado por un servo.